In [16]:
import os

PATH = '/home/stadlerm/Development/Projects/DreamChallenge/RA2_alpine_lads'

os.chdir(PATH)

In [2]:
from utils.config import Config

config = Config()

In [119]:
import os
import pandas as pd

training_csv = os.path.join(config.train_location, 'training.csv')

training_data = pd.read_csv(training_csv)

In [198]:
# Prepare outcome vector
y = np.zeros((training_data.shape[0], 4))

# Set the class of each sample for each outcome to the most rare outcome for this sample
def add_y_classes(values, y_idx):
    val, counts = np.unique(values, return_counts = True)
    
    sort_idx = np.argsort(counts)[::-1]
    
    for idx in sort_idx[1:]:
        min_val_idx = np.where(values == val[idx])
        min_val_idx = np.unique(min_val_idx[0])
        
        if(min_val_idx.size > 1):
            y[min_val_idx, y_idx] = val[idx]

In [200]:
# Regex for hand narrowing, erosion, and feet narrowing columns
column_reg = ['.*H_.*_J.*', '.*H_.*_E.*', '.*F_.*_J.*']

for idx, reg in enumerate(column_reg):
    p = re.compile(reg)

    column_idx = np.where([p.match(column) for column in training_data.columns])[0]
    columns = training_data.columns[column_idx]
    
    add_y_classes(training_data[columns].to_numpy(), idx)

# Process feet erosion differently because it goes from 0-10, and higher values are extremely rare (8-10)
p = re.compile('.*F_.*_E.*')
column_idx = np.where([p.match(column) for column in training_data.columns])[0]
columns = training_data.columns[column_idx]

values = np.round(training_data[columns] / 2)
# Very few have 5, so add them to 4
values[values == 5] = 4
add_y_classes(values, 3)

In [202]:
print(np.unique(y[:, 0], return_counts = True))
print(np.unique(y[:, 1], return_counts = True))
print(np.unique(y[:, 2], return_counts = True))
print(np.unique(y[:, 3], return_counts = True))

(array([0., 1., 2., 3., 4.]), array([163,  22,  65,  37,  81]))
(array([0., 1., 2., 3., 4., 5.]), array([145, 107,  45,  53,   4,  14]))
(array([0., 1., 2., 3., 4.]), array([257,   7,  47,  28,  29]))
(array([0., 1., 2., 3., 4.]), array([268,  66,  26,   4,   4]))


In [203]:
from sklearn.model_selection import train_test_split

train_idx, test_idx = train_test_split(np.arange(training_data.shape[0]), test_size = 0.2, stratify = y[:, 3])

In [204]:
print(np.unique(y[train_idx, 0], return_counts = True))
print(np.unique(y[train_idx, 1], return_counts = True))
print(np.unique(y[train_idx, 2], return_counts = True))
print(np.unique(y[train_idx, 3], return_counts = True))

(array([0., 1., 2., 3., 4.]), array([130,  17,  52,  31,  64]))
(array([0., 1., 2., 3., 4., 5.]), array([112,  91,  36,  42,   3,  10]))
(array([0., 1., 2., 3., 4.]), array([202,   7,  38,  25,  22]))
(array([0., 1., 2., 3., 4.]), array([214,  53,  21,   3,   3]))


In [205]:
print(np.unique(y[test_idx, 0], return_counts = True))
print(np.unique(y[test_idx, 1], return_counts = True))
print(np.unique(y[test_idx, 2], return_counts = True))
print(np.unique(y[test_idx, 3], return_counts = True))

(array([0., 1., 2., 3., 4.]), array([33,  5, 13,  6, 17]))
(array([0., 1., 2., 3., 4., 5.]), array([33, 16,  9, 11,  1,  4]))
(array([0., 2., 3., 4.]), array([55,  9,  3,  7]))
(array([0., 1., 2., 3., 4.]), array([54, 13,  5,  1,  1]))


In [209]:
train_patient_id = training_data.iloc[train_idx]['Patient_ID'].to_numpy()
test_patient_id = training_data.iloc[test_idx]['Patient_ID'].to_numpy()

In [210]:
print(train_patient_id)

['UAB411' 'UAB084' 'UAB462' 'UAB058' 'UAB039' 'UAB711' 'UAB139' 'UAB722'
 'UAB232' 'UAB179' 'UAB667' 'UAB562' 'UAB170' 'UAB135' 'UAB452' 'UAB379'
 'UAB081' 'UAB127' 'UAB477' 'UAB126' 'UAB407' 'UAB571' 'UAB669' 'UAB374'
 'UAB463' 'UAB124' 'UAB492' 'UAB106' 'UAB451' 'UAB035' 'UAB225' 'UAB099'
 'UAB509' 'UAB107' 'UAB700' 'UAB075' 'UAB661' 'UAB231' 'UAB078' 'UAB503'
 'UAB706' 'UAB490' 'UAB334' 'UAB694' 'UAB310' 'UAB216' 'UAB036' 'UAB018'
 'UAB488' 'UAB630' 'UAB297' 'UAB267' 'UAB196' 'UAB687' 'UAB640' 'UAB200'
 'UAB054' 'UAB022' 'UAB399' 'UAB405' 'UAB417' 'UAB199' 'UAB210' 'UAB609'
 'UAB277' 'UAB701' 'UAB543' 'UAB504' 'UAB280' 'UAB654' 'UAB395' 'UAB521'
 'UAB007' 'UAB337' 'UAB392' 'UAB345' 'UAB238' 'UAB319' 'UAB491' 'UAB071'
 'UAB707' 'UAB032' 'UAB160' 'UAB057' 'UAB645' 'UAB538' 'UAB497' 'UAB680'
 'UAB198' 'UAB065' 'UAB114' 'UAB163' 'UAB428' 'UAB168' 'UAB716' 'UAB195'
 'UAB500' 'UAB487' 'UAB394' 'UAB108' 'UAB159' 'UAB651' 'UAB040' 'UAB080'
 'UAB115' 'UAB145' 'UAB012' 'UAB393' 'UAB672' 'UAB1

In [211]:
print(test_patient_id)

['UAB597' 'UAB642' 'UAB573' 'UAB698' 'UAB717' 'UAB129' 'UAB030' 'UAB697'
 'UAB449' 'UAB307' 'UAB212' 'UAB342' 'UAB125' 'UAB427' 'UAB104' 'UAB133'
 'UAB413' 'UAB348' 'UAB100' 'UAB360' 'UAB499' 'UAB318' 'UAB595' 'UAB229'
 'UAB260' 'UAB574' 'UAB171' 'UAB472' 'UAB050' 'UAB070' 'UAB666' 'UAB061'
 'UAB675' 'UAB105' 'UAB592' 'UAB300' 'UAB251' 'UAB302' 'UAB177' 'UAB530'
 'UAB162' 'UAB501' 'UAB006' 'UAB181' 'UAB366' 'UAB046' 'UAB236' 'UAB522'
 'UAB569' 'UAB166' 'UAB625' 'UAB622' 'UAB683' 'UAB649' 'UAB041' 'UAB508'
 'UAB037' 'UAB063' 'UAB228' 'UAB536' 'UAB224' 'UAB464' 'UAB233' 'UAB690'
 'UAB270' 'UAB517' 'UAB243' 'UAB244' 'UAB634' 'UAB441' 'UAB033' 'UAB150'
 'UAB205' 'UAB436']


In [212]:
hands_dataframe = pd.read_csv('./data/hand_joint_data.csv')
feet_dataframe = pd.read_csv('./data/feet_joint_data.csv')

In [218]:
hand_idx = [image_name.split('-')[0] in test_patient_id for image_name in hands_dataframe['image_name']]
feet_idx = [image_name.split('-')[0] in test_patient_id for image_name in feet_dataframe['image_name']]

In [222]:
hands_dataframe.iloc[np.where(np.logical_not(hand_idx))[0]].to_csv('./data/hand_joint_data_train.csv')
feet_dataframe.iloc[np.where(np.logical_not(feet_idx))[0]].to_csv('./data/feet_joint_data_train.csv')

hands_dataframe.iloc[np.where(hand_idx)[0]].to_csv('./data/hand_joint_data_test.csv')
feet_dataframe.iloc[np.where(feet_idx)[0]].to_csv('./data/feet_joint_data_test.csv')